Arrange the dates you want and download [from here](https://analytics.google.com/analytics/web/#/savedreport/OqXu9BGgSZuBRBD01bg3yw/a41411988w90270749p93874080/_u.date00=20200904&_u.date01=20200910&_u.date10=20200501&_u.date11=20200507&_.advseg=&_.useg=&_.sectionId=&_.goalOption=2&_r.dsa=1&explorer-table.advFilter=%5B%5B0,%22analytics.landingPagePath%22,%22PT%22,%22~2Fblog~2F%22,0%5D,%5B1,%22analytics.visits%22,%22GT%22,%22200%22,0%5D,%5B0,%22analytics.sourceMedium%22,%22PT%22,%22google%20~2F%20organic%22,0%5D%5D&explorer-table.plotKeys=%5B%5D&explorer-table.secSegmentId=analytics.sourceMedium&explorer-table.rowStart=0&explorer-table.rowCount=100). 

If that link doesn't work, Behavior > Site content > Landing pages and then set up your time comparison, and filter by whatever source/medium you'd like and other filters you'd like. 

When you're done, export the report as a CSV, rename the CSV file `ga-export.csv`, and then place it in the same directory as the Jupyter Notebook you'll be using to run this code. Then, run the code below.

Alternately, you can just run all cells to generate a CSV that includes all three rate calculations as separate columns.

In [1]:
import pandas as pd

#read the CSV, skipping the first six rows to avoid the Google-added header
df = pd.read_csv('ga-export.csv', skiprows=6)
# drop rows with missing data, this eliminates dumb rows GA adds to the CSV
df = df.dropna()

# Defining the columns we want to keep
df = df[['Landing Page', 'Date Range', 'Sessions', 'New Users', 'Bounce Rate', 'Pages / Session', 'Avg. Session Duration','01: New Account Signups (Goal 2 Conversion Rate)', '01: New Account Signups (Goal 2 Completions)']]

#convert the dataframe to a list
dflist = df.values.tolist()

# count the rows and add even and odd rows to two separate lists
rowcounter = 0
evenlist = []
oddlist = []
for row in dflist:
    rowcounter += 1
    if rowcounter % 2 == 0:
        evenlist.append(row)
    else:
        oddlist.append(row)

# Turn the even and odd lists back into two separate dataframes and add column names for clarity
# Even numbers are the OLD (orange) data from GA, even is the CURRENT (blue) data
df_even = pd.DataFrame(evenlist,columns=['Page', 'Dates', 'Sessions', 'NewUsers', 'BounceRate', 'PagesPerSession', 'AvgSessionDuration', 'SignupRate', 'Signups'])
df_odd = pd.DataFrame(oddlist,columns=['Page', 'Dates_now', 'Sessions_now', 'NewUsers_now', 'BounceRate_now', 'PagesPerSession_now', 'AvgSessionDuration_now', 'SignupRate_now', 'Signups_now'])

# merge the even and odd dataframes into a new dataframe on the 'page column'
joined = pd.merge(df_odd, df_even, on='Page', how='outer')

# eliminate commas and percentage signs in the numbers for easier conversion to numberic datatypes
joined.replace(',','', regex=True, inplace=True)
joined.replace('%','', regex=True, inplace=True)

## % Change in Session Numbers

When the above cell has run, you can now run the below cell. As written, it will give you a list of the posts by percent change in sessions from biggest drop to biggest gain, and create a CSV file called `blog-analysis.csv` in the same directory as your notebook. It will also print the first three rows.

If you wish to reverse this sort order, change `ascending=True` to `ascending=False`.

If you wish to analyze a different factor, you can change the column names, or look below for other prewritten code cells.

If you wish to print more rows, change the number in `output.head(3)` to the number of rows you'd like print.

In [2]:
# calculate the percent change 
joined['SessionDelta'] = (joined['Sessions_now'].astype('float') - joined['Sessions'].astype('float')) / joined['Sessions'].astype('float') * 100
# sort by the percent change column and save in a new dataframe called output
output = joined.sort_values(by=['SessionDelta'], ascending=True)

# export the dataframe as a CSV, not including the index numbers
output.to_csv('blog-analysis.csv', index=False)
output.head(3)

,Page,Dates_now,Sessions_now,NewUsers_now,BounceRate_now,PagesPerSession_now,AvgSessionDuration_now,SignupRate_now,Signups_now,Dates,Sessions,NewUsers,BounceRate,PagesPerSession,AvgSessionDuration,SignupRate,Signups,SessionDelta
46,www.dataquest.io/blog/python-json-tutorial/,Sep 4 2020 - Sep 10 2020,493,358,85.40,1.45,00:07:46,0.00,0.0,May 1 2020 - May 7 2020,1362,941,9.10,2.27,00:07:25,0.15,2.0,-63.803231
59,www.dataquest.io/blog/docker-data-science/,Sep 4 2020 - Sep 10 2020,334,239,88.92,1.14,00:09:47,0.00,0.0,May 1 2020 - May 7 2020,792,555,6.57,2.46,00:10:28,0.25,2.0,-57.828283
26,www.dataquest.io/blog/web-scraping-beautifulsoup/,Sep 4 2020 - Sep 10 2020,875,551,78.17,1.50,00:11:00,0.34,3.0,May 1 2020 - May 7 2020,2025,1335,5.78,2.50,00:10:27,0.15,3.0,-56.790123


## % Change in Signups

Run the below code to generate a column for % change in signups over time. 

In [3]:
joined['SignupDelta'] = (joined['Signups_now'].astype('float') - joined['Signups'].astype('float')) / joined['Signups'].astype('float') * 100
output = joined.sort_values(by=['SignupDelta'], ascending=True)

output.to_csv('blog-analysis.csv', index=False)
output.head(3)

,Page,Dates_now,Sessions_now,NewUsers_now,BounceRate_now,PagesPerSession_now,AvgSessionDuration_now,SignupRate_now,Signups_now,Dates,Sessions,NewUsers,BounceRate,PagesPerSession,AvgSessionDuration,SignupRate,Signups,SessionDelta,SignupDelta
46,www.dataquest.io/blog/python-json-tutorial/,Sep 4 2020 - Sep 10 2020,493,358,85.40,1.45,00:07:46,0.00,0.0,May 1 2020 - May 7 2020,1362,941,9.10,2.27,00:07:25,0.15,2.0,-63.803231,-100.0
18,www.dataquest.io/blog/sql-insert-tutorial/,Sep 4 2020 - Sep 10 2020,1344,879,84.90,1.21,00:09:09,0.00,0.0,May 1 2020 - May 7 2020,1708,1040,6.44,2.36,00:09:48,0.06,1.0,-21.311475,-100.0
59,www.dataquest.io/blog/docker-data-science/,Sep 4 2020 - Sep 10 2020,334,239,88.92,1.14,00:09:47,0.00,0.0,May 1 2020 - May 7 2020,792,555,6.57,2.46,00:10:28,0.25,2.0,-57.828283,-100.0


## % Change in Signup Rate

Run the below code to generate a column for % change in signup rate over time.

In [4]:
joined['SignupRateDelta'] = (joined['SignupRate_now'].astype('float') - joined['SignupRate'].astype('float')) / joined['SignupRate'].astype('float') * 100
output = joined.sort_values(by=['SignupRateDelta'], ascending=True)

output.to_csv('blog-analysis.csv', index=False)
output.head(3)

,Page,Dates_now,Sessions_now,NewUsers_now,BounceRate_now,PagesPerSession_now,AvgSessionDuration_now,SignupRate_now,Signups_now,Dates,Sessions,NewUsers,BounceRate,PagesPerSession,AvgSessionDuration,SignupRate,Signups,SessionDelta,SignupDelta,SignupRateDelta
18,www.dataquest.io/blog/sql-insert-tutorial/,Sep 4 2020 - Sep 10 2020,1344,879,84.90,1.21,00:09:09,0.00,0.0,May 1 2020 - May 7 2020,1708,1040,6.44,2.36,00:09:48,0.06,1.0,-21.311475,-100.0,-100.0
59,www.dataquest.io/blog/docker-data-science/,Sep 4 2020 - Sep 10 2020,334,239,88.92,1.14,00:09:47,0.00,0.0,May 1 2020 - May 7 2020,792,555,6.57,2.46,00:10:28,0.25,2.0,-57.828283,-100.0,-100.0
62,www.dataquest.io/blog/python-for-loop-tutorial/,Sep 4 2020 - Sep 10 2020,292,228,84.25,1.21,00:07:01,0.00,0.0,May 1 2020 - May 7 2020,242,169,6.61,2.85,00:12:47,1.24,3.0,20.661157,-100.0,-100.0
